<a href="https://colab.research.google.com/github/KubX44/Tp-Spark-GraphX-et-GraphFrames/blob/main/TP6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=c69ead13442db385d293e1a340bf41e67f4a1e9728ff701d5755e16f0cb517b5
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


1) Créer un DataFrame pour chaque fichier

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = (SparkSession
            .builder
            .config('spark.jars.packages', 'graphframes:graphframes:0.8.2-spark3.2-s_2.12')
            .appName("tp6")
            .getOrCreate()
)

In [4]:
Stations = spark.read.option("header","true")\
.csv("/content/station_data.csv")
Trips = spark.read.option("header","true")\
.csv("/content/trip_data.csv")

2) Dans la table des sommets (le DataFrame qui représente le fichier station_data.csv),
renommer la colonne name en id.

In [13]:
stationsWId = Stations.withColumnRenamed("name", "id").distinct()

3) Dans la table des arcs (le DataFrame qui représente le fichier trip_data.csv), renommer
la colonne Start Station en src et la colonne End Station en dst.

In [11]:
tripSrcDst = Trips\
.withColumnRenamed("Start Station", "src")\
.withColumnRenamed("End Station", "dst")

4) Créer un GraphFrame, qui représente notre graphe.

In [14]:
from graphframes import GraphFrame
stationGraph = GraphFrame(stationsWId, tripSrcDst)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


5) Retourner le nombre de voyages effectués entre chaque source et destination triés par
ordre décroissant (selon le nombre de voyages).


In [15]:
from pyspark.sql.functions import desc
stationGraph.edges.groupBy("src", "dst")\
.count().orderBy(desc("count"))\
.show(10, False)

+---------------------------------------+----------------------------------------+-----+
|src                                    |dst                                     |count|
+---------------------------------------+----------------------------------------+-----+
|San Francisco Caltrain 2 (330 Townsend)|Townsend at 7th                         |4    |
|5th at Howard                          |San Francisco Caltrain 2 (330 Townsend) |3    |
|San Francisco Caltrain 2 (330 Townsend)|Powell Street BART                      |2    |
|2nd at Townsend                        |Market at Sansome                       |2    |
|Spear at Folsom                        |2nd at Townsend                         |2    |
|Market at Sansome                      |Broadway St at Battery St               |2    |
|Davis at Jackson                       |Embarcadero at Sansome                  |2    |
|San Francisco Caltrain 2 (330 Townsend)|2nd at South Park                       |2    |
|Steuart at Market   

6) Retourner le nombre de voyages qui se commencent ou se terminent à partir de la
station ‘Townsend at 7th’ triés par ordre décroissant (selon le nombre de voyages).

In [16]:
stationGraph.edges\
.where("src = 'Townsend at 7th' OR dst = 'Townsend at 7th'")\
.groupBy("src", "dst").count()\
.orderBy(desc("count"))\
.show(10)

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|San Francisco Cal...|     Townsend at 7th|    4|
|       5th at Howard|     Townsend at 7th|    1|
|     Townsend at 7th|     Spear at Folsom|    1|
|     Townsend at 7th|Harry Bridges Pla...|    1|
|     Spear at Folsom|     Townsend at 7th|    1|
|       Howard at 2nd|     Townsend at 7th|    1|
+--------------------+--------------------+-----+



7) Retourner les sommets qui n'ont jamais été une destination d'un voyage qui commence
à partir de ' Spear at Folsom'.

In [17]:
# Requête avec la fonction subtract
SpearFolsomDestinations = stationGraph \
    .edges \
    .where("src = 'Spear at Folsom'") \
    .select("dst") \
    .distinct()

otherStations = stationGraph \
    .edges \
    .select("dst") \
    .distinct() \
    .subtract(SpearFolsomDestinations)

# Affichage des résultats
otherStations.show(1000, False)

+---------------------------------------------+
|dst                                          |
+---------------------------------------------+
|Powell at Post (Union Square)                |
|Market at Sansome                            |
|Spear at Folsom                              |
|San Salvador at 1st                          |
|Rengstorff Avenue / California Street        |
|Civic Center BART (7th at Market)            |
|Mountain View City Hall                      |
|Market at 4th                                |
|Temporary Transbay Terminal (Howard at Beale)|
|Steuart at Market                            |
|San Francisco City Hall                      |
|South Van Ness at Market                     |
|Embarcadero at Bryant                        |
|Mountain View Caltrain Station               |
|Powell Street BART                           |
|Clay at Battery                              |
|Market at 10th                               |
|Broadway St at Battery St              

8) Retourner la station qui a le nombre maximum de voyages entrants.


In [18]:
from pyspark.sql.functions import desc
stationGraph.inDegrees.orderBy(desc("inDegree")).limit(1)\
.show(1, False)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+---------------------------------------+--------+
|id                                     |inDegree|
+---------------------------------------+--------+
|San Francisco Caltrain 2 (330 Townsend)|9       |
+---------------------------------------+--------+



9) Retourner le voyage qui a la plus grande durée

In [19]:
stationGraph.edges.orderBy(desc("Duration")).limit(1).show()

+-------+--------+---------------+--------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|           src|Start Terminal|       End Date|                 dst|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913448|     932|8/31/2015 21:57|Post at Kearny|            47|8/31/2015 22:12|South Van Ness at...|          66|   472|     Subscriber|   94702|
+-------+--------+---------------+--------------+--------------+---------------+--------------------+------------+------+---------------+--------+



10)Créer un sous-graphe qui ne contient que les voyages qui se commencent ou se
terminent à ‘Townsend at 7th’.

In [20]:
townAnd7thEdges = stationGraph.edges\
.where("src = 'Townsend at 7th' OR dst = 'Townsend at 7th'")

subgraph = GraphFrame(stationGraph.vertices, townAnd7thEdges)

11) Retourner tous les chemins qui forment un motif en "triangle" entre trois stations.

In [21]:
motifs = stationGraph.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)")
motifs.show(1000, False)

+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+
|a                                                                                               |ab                                                                                                                                         |b                                                

12) Retourner tous les chemins qui passent par trois sommets et qui commencent à partir de 'Townsend at 7th'

In [22]:
# Recherche de tous les chemins qui passent par trois sommets
paths = stationGraph.find("(a)-[]->(b); (b)-[]->(c)")

# Filtrage des chemins qui commencent à partir de 'Townsend at 7th'
filtered_paths = paths.filter("a.id = 'Townsend at 7th'")

# Affichage des résultats
filtered_paths.show(truncate=False)

+---------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|a                                                                          |b                                                                                               |c                                                                                                  |
+---------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|{65, Townsend at 7th, 37.771058, -122.402717, 15, San Francisco, 8/22/2013}|{49, Spear at Folsom, 37.790302, -122.390637, 19, San Francisco, 8/20/2013}                     |{